<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/example_predict_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/koyomin9zx/UITQA-Vietnamese-Question-Answering.git

In [ ]:
!pip install pytorch-transformers
!pip install underthesea 
!pip install unidecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ln -s /content/drive/'My Drive'/data
%cd /content/UITQA-Vietnamese-Question-Answering

In [29]:
# importing libraries 
from underthesea import word_tokenize, sent_tokenize, pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial import distance 
from collections import defaultdict, OrderedDict 
from string import punctuation
import re
import glob
from bert import QA
from utils import *
import time
import sys
import os

In [8]:
start = time.time()
model=QA('/content/data/BERT_Squad_WIki_UIT_pretrain') #path to model
end = time.time()
print("time load model: "+str(round((end - start),2)))

time load model: 38.36


In [34]:
def load_data(path):
  data=[]
  all_files = glob.glob(path + "/*.txt")
  for file in all_files:
      passage=open(file, "r", encoding='utf-8').read()
      data.append(passage)
  return data

def vi_tokenizer(row):
    return word_tokenize(row, format="text")

def remove_stopwords(stopwords,text):
  sent = [s for s in text.split() if s not in stopwords ]
  sent = ' '.join(sent)
  return sent


def standardize_data(df,stopwords):
    hl_cleansed=[]
    remove = punctuation
    pattern = "[{}]".format(remove) # create the pattern
    re_space=re.compile('\s+')
    re_trailing=re.compile('^\s+|\s+?$')
    for row in df:
        row=re.sub(pattern, " ", row) 
        row=re.sub(re_space,' ',row)
        row=re.sub(re_trailing,' ',row)
        row = row.strip()
        row = remove_stopwords(stopwords,row)
        hl_cleansed.append(row)
    return hl_cleansed

def keywords_extraction(sent):
  rs=""
  for i in pos_tag(sent):
    if i[1] !='P' and i[1] != 'CH':
      rs=rs+' '+i[0]
  return rs.strip()


## Converting 3D array of array into 1D array 
def arr_convert_1d(arr): 
    arr = np.array(arr) 
    arr = np.concatenate( arr, axis=0 ) 
    arr = np.concatenate( arr, axis=0 ) 
    return arr 
  
## Cosine Similarity 
def cosine(trans): 
    cos = [] 
    cos.append(cosine_similarity(trans[0], trans[1])) 
    return cos

def tfidf(str1, str2,tf_idf_vetor): 
    corpus = [str1,str2] 
    trans = tf_idf_vetor.transform(corpus)
    cos=cosine(trans) 
    return arr_convert_1d(cos)[0]

def relevance_ranking(query,data,vect,top_n_matching=None):
  key_words_query=keywords_extraction(query)
  results=[]
  for doc in data:
    rs=dict()
    ir_score=tfidf(key_words_query, doc,vect)
    num_overlap=len(set(key_words_query.split()) & set(doc.split()))
    if ir_score!=0.0:
      rs['overlap_key_words']=num_overlap
      rs['ir_score']=ir_score
      rs['content']=doc
      results.append(rs)
  results=sorted(results, key=lambda k: k['overlap_key_words'], reverse=True)
  if top_n_matching:
    results=results[:top_n_matching]
  return results

In [31]:
data=load_data('/content/data/data_QA/data')
stopwords = set(open('/content/data/data_QA/stopwords/stopwords.txt').read().split(' ')[:-1])


In [ ]:
#Building index
data_standard=standardize_data(data,stopwords)
vect = TfidfVectorizer(min_df=1, max_df=0.8,max_features=8000,sublinear_tf=True,ngram_range=(1,3)) 
vect.fit(data_standard)

In [35]:
rs=relevance_ranking('Võ Nguyên Giáp quê ở đâu ?',data_standard,vect,1)
rs

[{'content': 'Võ Nguyên Giáp sinh 1911 Võ Nguyên Giáp sinh 1911 nhà hoạt động nổi tiếng Đảng Cộng sản Nhà nước Việt Nam nhà huy nhà lí luận quân xuất sắc Việt Nam Quê xã Lộc Thuỷ huyện Lệ Thuỷ tỉnh Quảng Bình Sớm hoạt động cách mạng tham gia lãnh đạo phong trào học sinh Huế 1925 tham gia Tân Việt Cách mạng Đảng Bị thực dân Pháp bắt 1930 đưa về quê quản thúc Ra Hà Nội tham gia hoạt động trong phong trào Mặt trận dân chủ Tham gia sáng lập báo Lao động Tiếng nói chúng ta biên tập báo Tin tức Dân chúng Chủ tịch Uỷ ban Báo chí Bắc Kỳ trong phong trào Đông Dương đại hội Gia nhập Đảng Cộng sản Đông Dương 1940 Gây dựng cơ sở cách mạng tổ chức Việt Minh ở Cao Bằng 1941 tổ chức Ban Xung phong Nam tiến 1942 Được lãnh tụ Nguyễn Ái Quốc giao tổ chức Đội Việt Nam Tuyên truyền Giải phóng quân 12 1944 huy đội đánh thắng hai trận đầu ở Phai Khắt 25 12 1944 Nà Ngần 26 12 1944 Uỷ viên Uỷ ban Quân Cách mạng Bắc Kỳ tư lệnh Việt Nam Giải phóng quân uỷ viên Ban Chỉ huy Lâm thời khu giải phóng Việt Bắc tham g

In [36]:
def IR_QA(query,data,top_n_matching=None):
  key_words_query=keywords_extraction(query)
  IR_ranking=relevance_ranking(key_words_query,data,vect,top_n_matching)
  rs=[]
  for ranking in IR_ranking :
    result=dict()
    answer = model.predict(ranking['content'],query)
    if answer['confidence'] >0.4:
      result['overlap_key_words']=ranking['overlap_key_words']
      result['answer']=answer['answer']
      result['ir_score']=ranking['ir_score']
      result['bert_score']=answer['confidence']
      result['content']=ranking['content']
      rs.append(result)
  return rs 


In [42]:
query="Lương Võ Đế sai Trần Bá Tiên đem quân sang đánh phá nước ta năm nào?"

IR_QA(query,data,3)

[{'answer': '545',
  'bert_score': 0.7302391759904686,
  'content': 'Triệu Quang Phục (…- Tân Mão)\n\nDanh tướng nhà Tiền Lý, con Thái phó Triệu Túc, quê ở Châu Biên, phủ Vĩnh Tường, tỉnh Vĩnh Yên (nay thuộc Vĩnh Phú).\n\nTừ năm Tân Dậu 541 ông theo cha ra giúp Lý Bôn đánh đuổi quân nhà Lương xâm lược nước ta. Năm Giáp Tí 544, khởi nghĩa thành công, Lý Bôn xưng đế, phong ông làm Tả tướng quân.\n\nNăm Ất Sửu 545, Lương Võ Đế lại sai Trần Bá Tiên đem quân sang đánh phá, đưa Diêu Phiêu làm Thứ sử\n\nNăm 546, Lý Bí phải rút quân về động Khuất Lão (Tam Nông, Phú Thọ ngày nay; có ý kiến cho rằng, Khuất Lão thuộc Tuyên Quang), giao binh cho Triệu Quang Phục. Ông lấy vùng Đầm Dạ Trạch (Khoái Châu, Hưng Yên ngày nay) làm căn cứ. Nhân dân gọi ông là Dạ Trạch Vương.\n\nNăm 550, ông tập trung lực lượng đánh bại quân Lương lấy lại thành Long Biên (nay thuộc Bắc Ninh). Quân sĩ suy tôn ông làm vua tức Triệu Việt Vương.\n\nNăm 557, Lý Phật Tử (cùng họ với Lý Bí) kéo quân về đánh, đòi ngôi vua. Sau nhi